### data merge working

In [74]:
import os
import pandas as pd

In [77]:
df = pd.read_csv("도곡로_통행속도_기상_지하철.csv", encoding='cp949')

print(df.head())

           일자 요일  도로명       링크아이디       시점명       종점명  방향   거리  차선수  기능유형구분  \
0  2021-01-01  금  도곡로  1220020800       한티역    영동세브란스  하행  592    3  보조간선도로   
1  2021-01-01  금  도곡로  1220016700  싸리고개공원입구  도곡1동주민센터  상행  551    3  보조간선도로   
2  2021-01-01  금  도곡로  1220015400  싸리고개공원입구     뱅뱅사거리  하행  328    3  보조간선도로   
3  2021-01-01  금  도곡로  1220016600  도곡1동주민센터  싸리고개공원입구  하행  551    3  보조간선도로   
4  2021-01-01  금  도곡로  1220019000    영동세브란스  도곡1동주민센터  하행  554    3  보조간선도로   

   ... 강수량(mm) 풍속(m/s) 풍향(16방위)  적설(cm)  seq  dateKind isHoliday  dateName  \
0  ...     NaN     1.8    250.0     NaN  1.0       1.0      True      1월1일   
1  ...     NaN     1.8    250.0     NaN  1.0       1.0      True      1월1일   
2  ...     NaN     1.8    250.0     NaN  1.0       1.0      True      1월1일   
3  ...     NaN     1.8    250.0     NaN  1.0       1.0      True      1월1일   
4  ...     NaN     1.8    250.0     NaN  1.0       1.0      True      1월1일   

   승차  하차  
0   2  77  
1   2  77  
2   2  77  
3   2  7

In [75]:
speed22 = pd.read_excel("2022_강남_통행속도.xlsx")

In [50]:
speed22.head()

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20210101,금,강남대로,1220021100,우성아파트,강남역,상행,586,4,주간선도로,...,23.70,23.10,22.37,23.97,25.02,27.92,25.39,24.02,26.26,28.92
1,20210101,금,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,4,주간선도로,...,30.60,30.30,31.41,29.72,32.13,31.31,30.27,32.19,25.23,24.26
2,20210101,금,개포로,1220020500,대청초교,대청역,상행,862,2,기타도로,...,25.29,27.74,21.92,20.40,22.55,25.91,27.75,26.05,26.61,29.05
3,20210101,금,개포로,1220018600,대청역,수서경찰서,상행,318,2,기타도로,...,28.51,31.32,32.85,28.59,31.46,30.55,29.15,34.59,34.86,27.92
4,20210101,금,개포로,1220017200,수서경찰서,대모산입구역,상행,333,2,기타도로,...,24.25,19.70,24.65,27.11,29.45,23.17,29.77,23.95,28.42,31.99


In [51]:
def transform_time_data(df):
    # 시간 컬럼들 (01시 ~ 24시)
    time_columns = [f"{str(i).zfill(2)}시" for i in range(1, 25)]
    
    # 결과를 저장할 빈 리스트
    transformed_rows = []
    
    # 각 행에 대해 처리
    for _, row in df.iterrows():
        for time_col in time_columns:
            hour = time_col.replace('시', '')
            
            # 새로운 행 생성
            new_row = row.copy()
            # 시간 컬럼들 제거
            for tc in time_columns:
                new_row = new_row.drop(tc)
                
            # 일시 컬럼 추가
            date_str = str(row['일자'])  # YYYYMMDD 형식
            formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            new_row['일시'] = f"{formatted_date} {hour}:00:00"
            new_row['속도'] = row[time_col]  # 해당 시간의 속도값 저장
            
            transformed_rows.append(new_row)
    
    # 새로운 데이터프레임 생성
    result_df = pd.DataFrame(transformed_rows)
    return result_df

# 올림픽대로 데이터만 필터링하고 변환
olympic_df = speed22[speed22['도로명'].isin(['올림픽대로'])]
transformed_df = transform_time_data(olympic_df)

# 결과 정렬
transformed_df = transformed_df.sort_values('일시')

In [52]:
transformed_df.to_excel("22올림픽대로_통행속도.xlsx", index=False)

In [53]:
df = pd.read_excel('22올림픽대로_통행속도.xlsx')

In [54]:
df.tail()

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,권역구분,일시,속도
70075,20211231,금,올림픽대로,1210006400,동호대교남단,한남대교남단,상행,1155,4,도시고속도로,외곽,강남구,2021-12-31 24:00:00,78.33
70076,20211231,금,올림픽대로,1220005600,성수대교남단,동호대교남단,상행,1032,4,도시고속도로,외곽,강남구,2021-12-31 24:00:00,77.50
70077,20211231,금,올림픽대로,1220005500,영동대교남단,성수대교남단,상행,1996,4,도시고속도로,외곽,강남구,2021-12-31 24:00:00,82.92
70078,20211231,금,올림픽대로,1220004400,영동대교남단,올림픽대로동부간선JC,하행,1250,4,도시고속도로,외곽,강남구,2021-12-31 24:00:00,79.50
70079,20211231,금,올림픽대로,1230004500,올림픽대로동부간선JC,청담IC,하행,411,4,도시고속도로,외곽,강남구,2021-12-31 24:00:00,73.83


### 속도랑 기후자료 합치는 과정

In [67]:
def merge_speed_climat(speed_df, weather_df):

    # 두 데이터프레임의 '일시' 컬럼을 datetime 타입으로 변환
    speed_df['일시'] = pd.to_datetime(speed_df['일시'])
    weather_df['일시'] = pd.to_datetime(weather_df['일시'])

    # 데이터프레임 병합
    merged_df = pd.merge(speed_df, weather_df, on='일시', how='left')

    # 결과 저장
    merged_df.to_excel('21올림픽대로_통행속도_기상.xlsx', index=False)
    return merged_df

In [64]:
df.head()

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,권역구분,일시,속도
0,20210101,금,올림픽대로,1220004500,올림픽대로동부간선JC,영동대교남단,상행,1237,4,도시고속도로,외곽,강남구,2021-01-01 01:00:00,90.33
1,20210101,금,올림픽대로,1220005400,성수대교남단,영동대교남단,하행,1995,4,도시고속도로,외곽,강남구,2021-01-01 01:00:00,90.00
2,20210101,금,올림픽대로,1220004400,영동대교남단,올림픽대로동부간선JC,하행,1250,4,도시고속도로,외곽,강남구,2021-01-01 01:00:00,90.42
3,20210101,금,올림픽대로,1220005700,동호대교남단,성수대교남단,하행,1016,4,도시고속도로,외곽,강남구,2021-01-01 01:00:00,78.42
4,20210101,금,올림픽대로,1210006400,동호대교남단,한남대교남단,상행,1155,4,도시고속도로,외곽,강남구,2021-01-01 01:00:00,84.58


In [65]:
climate_21 = pd.read_excel("2021년 서울 기후 자료.xlsx")
climate_21.head()

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),적설(cm)
0,108,서울,2021-01-01 00:00:00,-8.3,NaN,1.8,250.0,NaN
1,108,서울,2021-01-01 01:00:00,-8.7,NaN,2.4,270.0,NaN
2,108,서울,2021-01-01 02:00:00,-9.1,NaN,1.6,270.0,NaN
3,108,서울,2021-01-01 03:00:00,-9.3,NaN,1.1,250.0,NaN
4,108,서울,2021-01-01 04:00:00,-9.3,NaN,0.3,0.0,NaN


In [76]:
pem = pd.read_csv("PEMS03_data.csv")
pem.head()

,313344,313349,313438,313450,313487,313493,313512,313551,313552,317752,...,313144,313159,313166,313172,313178,313184,313190,313197,313204,313339
0,20.0,20.0,182.0,182.0,91.0,182.0,136.0,91.0,91.0,136.0,...,158.0,63.0,63.0,63.0,125.0,114.0,63.0,63.0,115.0,63.0
1,22.0,22.0,174.0,174.0,87.0,174.0,131.0,87.0,87.0,131.0,...,158.0,62.0,62.0,62.0,144.0,109.0,62.0,63.0,109.0,62.0
2,22.0,22.0,183.0,183.0,92.0,183.0,139.0,92.0,92.0,139.0,...,160.0,57.0,57.0,57.0,131.0,116.0,57.0,57.0,115.0,57.0
3,50.0,49.0,137.0,139.0,60.0,158.0,111.0,57.0,55.0,99.0,...,174.0,109.0,30.0,58.0,106.0,102.0,52.0,111.0,88.0,48.0
4,37.0,35.0,128.0,123.0,54.0,131.0,111.0,46.0,52.0,78.0,...,162.0,77.0,31.0,55.0,109.0,127.0,49.0,109.0,76.0,47.0
